In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
import spacy

In [4]:
data = pd.read_csv("Restaurant reviews.csv")

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Restaurant  10000 non-null  object 
 1   Reviewer    9962 non-null   object 
 2   Review      9955 non-null   object 
 3   Rating      9962 non-null   object 
 4   Metadata    9962 non-null   object 
 5   Time        9962 non-null   object 
 6   Pictures    10000 non-null  int64  
 7   7514        1 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


In [6]:
data = data.drop(['Restaurant', 'Reviewer','Metadata', 'Time', 'Pictures', '7514'], axis=1)

In [7]:
data = data.dropna()

In [8]:
data = data[data.Rating != 'Like']

In [9]:
data.Rating.unique()

array(['5', '4', '1', '3', '2', '3.5', '4.5', '2.5', '1.5'], dtype=object)

In [10]:
data['Rating'] = data['Rating'].astype('float64')

In [11]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [12]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [13]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
tag_dict = {'J': wordnet.ADJ,'N': wordnet.NOUN,'V': wordnet.VERB,'R': wordnet.ADV}

In [14]:
#URLs present
for i in range(9954):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    if re.search(url_pattern, data.iloc[i,0]):
        print(i)

371
2596


In [15]:
import string
def preprocess_text(text):
    text = str(text).lower()
    clean_text = re.sub('https?://\S+|www\.\S+', '', text) #URLs
    clean_text = re.sub('[%s]' % re.escape(string.punctuation), '', clean_text) #Punctuations
    clean_text = re.sub('\n', '', clean_text) #Backslash n
    clean_text = [word for word in clean_text.split(' ') if word not in stop_words] #Remove stopwords
    clean_text=" ".join(clean_text)
    tokens = clean_text.split()
    pos_tags = nltk.pos_tag(tokens)
    wordnet_tags = [(token, tag_dict.get(tag[0].upper(), wordnet.NOUN)) for token, tag in pos_tags]
    tokens = [lemmatizer.lemmatize(token, tag) for token, tag in wordnet_tags] #Lemmatize
    return " ".join(tokens)

In [16]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [17]:
data['Clean_Review'] = data['Review'].apply(preprocess_text)

In [18]:
data['Clean_Review'][0]

'ambience good food quite good saturday lunch cost effective good place sate brunch one also chill friend parentswaiter soumen das really courteous helpful'

In [23]:
from transformers import pipeline

summarizer = pipeline("summarization", model="mabrouk/amazon-review-summarizer-bart")

In [68]:
review = data['Review'][50]
summarizer(review, min_length = 10, max_length = 20)

[{'summary_text': 'Pradeep and shubroto were amazing and friendly'}]

In [69]:
review

'Pradeep and shubroto were amazing and friendly. Service was awesome n food out of the world.must visit for all people out there. Will visit again'